In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/siamese/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 4000
start_epoch = 0
train_batch = 32
test_batch = 64
lr = 0.01
schedule = [500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/siamese' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = []

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(target_dir, '100_shot_style1')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])

val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_ = SiameseNetworkDataset(datasets.ImageFolder(val_target_dir), transform=val_aug, should_invert=False)
val_target_loader = DataLoader(val_target_, shuffle=True, num_workers=num_workers, batch_size=test_batch, pin_memory=True)
val_source_ = SiameseNetworkDataset(datasets.ImageFolder(val_source_dir), transform=val_aug, should_invert=False)
val_source_loader = DataLoader(val_source_, batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/siamese/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 4.17M


In [12]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [13]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [14]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += torch.norm(param, p=1)
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += torch.norm(param - teacher_model_weights[name], p=1)
    return sp_loss

# Loss

In [16]:
criterion = ContrastiveLoss(margin=2.0).cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [17]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

# Train

In [18]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
        
        outputs0 = student_model(inputs0)
        outputs1 = student_model(inputs1)
        
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)
        
        loss_main = criterion(outputs0, outputs1, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{total:.4f} | SPLoss:{eta:.4f} | | CLLoss:{sp:.4f} | top1:{tp1:.4f} | AUROC:{arc:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader),loss=loss, total=loss_main, eta=loss_sp, sp=loss_cls, tp1=top1.avg, arc=auroc))
    bar.finish()
    return (losses.avg, top1.avg)

In [19]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            loss_main = criterion(outputs0, outputs1, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_beta*loss_cls + sp_alpha*loss_sp

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())

            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                        batch=batch_idx + 1,
                        size=len(val_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=bar.elapsed_td,
                        eta=bar.eta_td,
                        loss=losses.avg,
                        top1=top1.avg,)
            bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:.4f} | AUROC:{arc:.4f}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg, arc=auroc))
    bar.finish()
    return (losses.avg, top1.avg)

In [20]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
#     if epoch in iter_time:
#         print("iterative training: feedback {}".format(epoch))
#         teacher_model.load_state_dict(student_model.state_dict())
#         teacher_model_weights = {}
#         for name, param in teacher_model.named_parameters():
#             teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, source_loss, train_acc, test_acc, source_acc, train_a])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
13/13 | Loss:329.6647 | MainLoss:1.9282 | SPLoss:2312.3291 | | CLLoss:965.0356 | top1:44.7917 | AUROC:0.4375
502/502 Data:0.009 | Batch:0.562 | Total:0:01:47 | ETA:0:00:00 | Loss:250.62532677005757 | top1:50.0903 | AUROC:0.4644
122/122 Data:0.000 | Batch:1.019 | Total:0:00:29 | ETA:0:00:00 | Loss:250.27906689766127 | top1:49.2564 | AUROC:1.0000

Epoch: [2 | 4000] LR: 0.010600
13/13 | Loss:252.3488 | MainLoss:2.0670 | SPLoss:2036.2588 | | CLLoss:466.5592 | top1:49.4792 | AUROC:0.7333

Epoch: [3 | 4000] LR: 0.011200
13/13 | Loss:244.5533 | MainLoss:2.8487 | SPLoss:2102.8794 | | CLLoss:314.1666 | top1:45.8333 | AUROC:0.5700

Epoch: [4 | 4000] LR: 0.011800
13/13 | Loss:255.7706 | MainLoss:2.1118 | SPLoss:2270.3555 | | CLLoss:266.2321 | top1:47.6562 | AUROC:0.4078

Epoch: [5 | 4000] LR: 0.012400
13/13 | Loss:272.7913 | MainLoss:1.7321 | SPLoss:2463.5464 | | CLLoss:247.0456 | top1:50.7812 | AUROC:0.6310

Epoch: [6 | 4000] LR: 0.013000
13/13 | Loss:294.2797 | M

13/13 | Loss:1163.6472 | MainLoss:2.0819 | SPLoss:10898.7998 | | CLLoss:716.8532 | top1:47.9167 | AUROC:0.6471

Epoch: [57 | 4000] LR: 0.040000
13/13 | Loss:1159.4476 | MainLoss:1.4662 | SPLoss:10864.7490 | | CLLoss:715.0649 | top1:51.5625 | AUROC:0.5833

Epoch: [58 | 4000] LR: 0.040000
13/13 | Loss:1158.2527 | MainLoss:2.4470 | SPLoss:10843.5654 | | CLLoss:714.4906 | top1:49.2188 | AUROC:0.6792

Epoch: [59 | 4000] LR: 0.040000
13/13 | Loss:1157.5026 | MainLoss:1.5943 | SPLoss:10837.9609 | | CLLoss:721.1217 | top1:52.6042 | AUROC:0.4939

Epoch: [60 | 4000] LR: 0.040000
13/13 | Loss:1159.0044 | MainLoss:2.0912 | SPLoss:10853.1709 | | CLLoss:715.9604 | top1:47.6562 | AUROC:0.4745

Epoch: [61 | 4000] LR: 0.040000
13/13 | Loss:1158.2606 | MainLoss:1.8405 | SPLoss:10847.3330 | | CLLoss:716.8687 | top1:52.0833 | AUROC:0.7020

Epoch: [62 | 4000] LR: 0.039999
13/13 | Loss:1155.8555 | MainLoss:1.8369 | SPLoss:10829.0186 | | CLLoss:711.1671 | top1:48.4375 | AUROC:0.6863

Epoch: [63 | 4000] LR: 0

13/13 | Loss:1159.6205 | MainLoss:1.8431 | SPLoss:10863.9766 | | CLLoss:713.7961 | top1:48.9583 | AUROC:0.5176

Epoch: [113 | 4000] LR: 0.039977
13/13 | Loss:1160.7064 | MainLoss:1.9485 | SPLoss:10871.2471 | | CLLoss:716.3316 | top1:53.9062 | AUROC:0.6914

Epoch: [114 | 4000] LR: 0.039976
13/13 | Loss:1161.0642 | MainLoss:2.4309 | SPLoss:10874.6592 | | CLLoss:711.6732 | top1:51.3021 | AUROC:0.6333

Epoch: [115 | 4000] LR: 0.039976
13/13 | Loss:1160.8137 | MainLoss:2.0957 | SPLoss:10877.2402 | | CLLoss:709.9407 | top1:47.3958 | AUROC:0.4118

Epoch: [116 | 4000] LR: 0.039975
13/13 | Loss:1161.6100 | MainLoss:2.1946 | SPLoss:10881.2080 | | CLLoss:712.9457 | top1:47.9167 | AUROC:0.6984

Epoch: [117 | 4000] LR: 0.039974
13/13 | Loss:1162.5027 | MainLoss:1.9322 | SPLoss:10890.7803 | | CLLoss:714.9249 | top1:47.3958 | AUROC:0.7305

Epoch: [118 | 4000] LR: 0.039973
13/13 | Loss:1163.0088 | MainLoss:2.1968 | SPLoss:10893.7246 | | CLLoss:714.3961 | top1:48.6979 | AUROC:0.6270

Epoch: [119 | 4000

13/13 | Loss:1176.6149 | MainLoss:2.2050 | SPLoss:11029.1768 | | CLLoss:714.9224 | top1:46.6146 | AUROC:0.5754

Epoch: [168 | 4000] LR: 0.039917
13/13 | Loss:1176.4613 | MainLoss:1.4619 | SPLoss:11043.4043 | | CLLoss:706.5890 | top1:49.2188 | AUROC:0.7083

Epoch: [169 | 4000] LR: 0.039916
13/13 | Loss:1177.3003 | MainLoss:2.1876 | SPLoss:11041.1709 | | CLLoss:709.9562 | top1:47.3958 | AUROC:0.7103

Epoch: [170 | 4000] LR: 0.039914
13/13 | Loss:1177.2926 | MainLoss:2.0775 | SPLoss:11040.8027 | | CLLoss:711.3471 | top1:48.1771 | AUROC:0.6353

Epoch: [171 | 4000] LR: 0.039913
13/13 | Loss:1178.3477 | MainLoss:1.9742 | SPLoss:11053.2773 | | CLLoss:710.4578 | top1:49.7396 | AUROC:0.5234

Epoch: [172 | 4000] LR: 0.039911
13/13 | Loss:1178.3456 | MainLoss:1.4557 | SPLoss:11058.5039 | | CLLoss:710.3944 | top1:50.2604 | AUROC:0.6208

Epoch: [173 | 4000] LR: 0.039910
13/13 | Loss:1179.4630 | MainLoss:2.0835 | SPLoss:11064.2920 | | CLLoss:709.5034 | top1:45.5729 | AUROC:0.4863

Epoch: [174 | 4000

Traceback (most recent call last):
  File "/home/cutz/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/cutz/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/cutz/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/cutz/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


13/13 | Loss:1185.0535 | MainLoss:1.5874 | SPLoss:11123.3281 | | CLLoss:711.3315 | top1:48.4375 | AUROC:0.6194

Epoch: [195 | 4000] LR: 0.039874
13/13 | Loss:1184.1786 | MainLoss:2.0760 | SPLoss:11116.9092 | | CLLoss:704.1169 | top1:53.6458 | AUROC:0.6980

Epoch: [196 | 4000] LR: 0.039872
13/13 | Loss:1185.5491 | MainLoss:1.3477 | SPLoss:11133.2881 | | CLLoss:708.7251 | top1:48.1771 | AUROC:0.5411

Epoch: [197 | 4000] LR: 0.039870
13/13 | Loss:1185.9016 | MainLoss:1.9505 | SPLoss:11133.9180 | | CLLoss:705.5930 | top1:52.8646 | AUROC:0.6523

Epoch: [198 | 4000] LR: 0.039869
13/13 | Loss:1185.7712 | MainLoss:1.7131 | SPLoss:11134.9375 | | CLLoss:705.6437 | top1:51.3021 | AUROC:0.4444

Epoch: [199 | 4000] LR: 0.039867
13/13 | Loss:1187.1375 | MainLoss:2.3242 | SPLoss:11143.1357 | | CLLoss:704.9958 | top1:48.6979 | AUROC:0.7004

Epoch: [200 | 4000] LR: 0.039865
13/13 | Loss:1185.8939 | MainLoss:2.1945 | SPLoss:11129.1182 | | CLLoss:707.8759 | top1:49.2188 | AUROC:0.5754

Epoch: [201 | 4000